# Exercise Sheet 6 

## Exercise 11 

A biometric system for access control has these benchmark data:
+ The system recognises an authorised person in 95 of 100 cases correctly
+ In 1 of 100 cases yet an unauthorised person is granted access.

The system is used to grant access to a server room for only 3 persons in a company
with 500 employees. Find the probability that an access granted person is indeed an
authorised person

In [ ]:
import numpy as np
import scipy.stats as stats

### a) by an analytic calculation and

In [ ]:
n_employees = 500
n_access = 3
p_access_correct = 95/100
p_access_incorrect = 1/100

In [ ]:
# a) code HERE
# Bayes: path of interest/all possible paths
p_access_authorized = (n_access / n_employees * p_access_correct) / (
        (n_access / n_employees * p_access_correct) + ((n_employees - n_access) / n_employees * p_access_incorrect))

print('Probability that an access granted person is indeed an authorised person: {:.2f}%'.format(
    p_access_authorized * 100))

### b) by estimating it by a relative frequency in a simulation

In [ ]:
def demand_authorized():
    dist = stats.multinomial(n=1, p=[1 - p_access_correct, p_access_incorrect])
    return int(np.where(dist.rvs(1) == 1)[1])  #0 for no access 1 for access

In [ ]:
def demand_unauthorized():
    dist = stats.multinomial(n=1, p=[1 - p_access_incorrect, p_access_incorrect])
    return int(np.where(dist.rvs(1) == 1)[1])  #0 for no access 1 for access

In [ ]:
def simu():
    """"
    0: For authorized and access
    1: For authorized no access
    2: For unauthorized no access
    3: For unauthorized and access
    """
    result = np.zeros(n_employees)

    for i in range(n_employees):
        #authorized employees
        if i < n_access:
            access = bool(demand_authorized())
            #if an authorized person got access write one, otherwise stays 0 like initialized
            if not access:
                result[i] = 1
            continue
        access = bool(demand_unauthorized())
        if not access:
            result[i] = 2
        else:
            result[i] = 3

    return result

In [ ]:
def control(times):
    sample = [simu() for _ in range(times)]
    values, counts = np.unique(sample, return_counts=True)
    return counts

In [ ]:
workingdays_per_year = 250
trails = workingdays_per_year
counts = control(trails)
prob = counts[0]/(counts[0]+counts[3])
print('Probability that an access granted person is indeed an authorised person: {:.2f}%'.format(
    prob * 100 ))

## Exercise 20

Use exercise 11 (biometric access system) and compute the 99% confidence interval.
How many access trials are necessary in order to judge the access system with the
given CI with a 2% accuracy? Determine the number of simulations for an adequate
accuracy and graphically visualize the confidence interval in dependency of the access
trials.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

# Access trials
n = np.linspace(100, 10000, 1000)

# See lecture theorem 3.2.2
# alpha = 0.01, ppf calculates phi inv
ci = norm.ppf(1-(0.01/2)) / (2 * np.sqrt(n))

plt.plot(ci, n)
plt.axvline(0.02, color='r')
plt.title("99% CI")
plt.xlabel("CI")
plt.ylabel("Number of Trials")
plt.show()

n_target = (norm.ppf(1-(0.01/2))/ (0.02 * 2)) ** 2
print('Number of Trials for a CI of 2%: {:.2f}'.format(n_target))